In [44]:
from sentence_transformers import SentenceTransformer, util
import pm4py
from pm4py.objects.bpmn.obj import BPMN
import torch
import torch.nn.functional as F
import numpy as np
import pandas as pd

In [45]:
bpmn = pm4py.read_bpmn('R_j04_model.bpmn')
temp=[]
for flow in bpmn.get_flows():
    temp.append([flow.source,flow.target,flow.get_name()])

for rel in temp:
    if(isinstance(rel[1],BPMN.Gateway) and len(rel[1].get_in_arcs())>1):
        rel[1]=rel[1].get_out_arcs()[0].target
        while(isinstance(rel[1],BPMN.Gateway)):
            rel[1]=rel[1].get_out_arcs()[0].target

temp1=[]
for rel in temp:
    if(rel[0]==rel[1]):
        continue
    elif(isinstance(rel[0],BPMN.Gateway) and len(rel[0].get_in_arcs())>1):
        continue
    else:
        temp1.append(rel)
df_BPMN = pd.DataFrame(columns=['Source', 'Target', 'Relation'])
for rel in temp1:
    if(isinstance(rel[1],BPMN.EndEvent) or isinstance(rel[0],BPMN.StartEvent)):
        continue
    if(isinstance(rel[0],BPMN.ExclusiveGateway)):
        # temp2.append([rel[0].get_in_arcs()[0].source, rel[1]])
        df_BPMN = df_BPMN.append({'Source': rel[0].get_in_arcs()[0].source.name, 'Target': rel[1].name, 'Relation': 'XOR_gateway'}, ignore_index=True)
        df_BPMN = df_BPMN.append({'Source': rel[2], 'Target': rel[1].name, 'Relation': 'XOR_condition'}, ignore_index=True)
        # print([rel[0].get_in_arcs()[0].source, rel[1]])
    elif(isinstance(rel[1],BPMN.ExclusiveGateway)):
        continue
    elif(isinstance(rel[0],BPMN.ParallelGateway)):
        df_BPMN = df_BPMN.append({'Source': rel[0].get_in_arcs()[0].source.name, 'Target': rel[1].name, 'Relation': 'parallel_gateway'}, ignore_index=True)
    elif(isinstance(rel[1],BPMN.ParallelGateway)):
        continue
    else:
        df_BPMN = df_BPMN.append({'Source': rel[0].name, 'Target': rel[1].name, 'Relation': 'Flow'}, ignore_index=True)

In [46]:
# log = pm4py.read_xes("R_j04_event.xes")
# bpmn_event = pm4py.discover_bpmn_inductive(log, activity_key='concept:name', case_id_key='case:concept:name', timestamp_key='time:timestamp')
bpmn_event = pm4py.read_bpmn('R_j04_event.bpmn')
temp=[]
for flow in bpmn_event.get_flows():
    temp.append([flow.source,flow.target,flow.get_name()])
temp1=[]
for rel in temp:
    if(isinstance(rel[1],BPMN.Gateway) and len(rel[1].get_in_arcs())>1):
        temp1.append([rel[0], rel[1].get_out_arcs()[0].target, rel[2]])
        # print([rel[0],rel[1].get_out_arcs()[0].target])
    elif(isinstance(rel[0],BPMN.Gateway) and len(rel[0].get_in_arcs())>1):
        continue
    else:
        temp1.append(rel)
df_BPMN_event = pd.DataFrame(columns=['Source', 'Target', 'Relation'])
for rel in temp1:
    if(isinstance(rel[1],BPMN.EndEvent) or isinstance(rel[0],BPMN.StartEvent)):
        continue
    if(isinstance(rel[0],BPMN.ExclusiveGateway)):
        # temp2.append([rel[0].get_in_arcs()[0].source, rel[1]])
        df_BPMN_event = df_BPMN_event.append({'Source': rel[0].get_in_arcs()[0].source.name, 'Target': rel[1].name, 'Relation': 'XOR_gateway'}, ignore_index=True)
        df_BPMN_event = df_BPMN_event.append({'Source': rel[2], 'Target': rel[1].name, 'Relation': 'XOR_condition'}, ignore_index=True)
        # print([rel[0].get_in_arcs()[0].source, rel[1]])
    elif(isinstance(rel[1],BPMN.ExclusiveGateway)):
        continue
    elif(isinstance(rel[0],BPMN.ParallelGateway)):
        df_BPMN_event = df_BPMN_event.append({'Source': rel[0].get_in_arcs()[0].source.name, 'Target': rel[1].name, 'Relation': 'parallel_gateway'}, ignore_index=True)
    elif(isinstance(rel[1],BPMN.ParallelGateway)):
        continue
    else:
        df_BPMN_event = df_BPMN_event.append({'Source': rel[0].name, 'Target': rel[1].name, 'Relation': 'Flow'}, ignore_index=True)

In [47]:
list_event=[]
for row in df_BPMN_event.itertuples(index=False):
    if(row.Relation!='XOR_condition'):
        list_event.append((row.Source,row.Target,row.Relation))
list_BPMN=[]
for row in df_BPMN.itertuples(index=False):
    if(row.Relation!='XOR_condition'):
        list_BPMN.append((row.Source,row.Target,row.Relation))
dif_gold=set(list(set(list_BPMN).difference(set(list_event)))+list(set(list_event).difference(set(list_BPMN))))

In [48]:
log = pm4py.read_xes("R_j04_event.xes")
bpmn_event = pm4py.discover_bpmn_inductive(log, activity_key='concept:name', case_id_key='case:concept:name', timestamp_key='time:timestamp')
# bpmn_event = pm4py.read_bpmn('R_j04_event.bpmn')
temp=[]
for flow in bpmn_event.get_flows():
    temp.append([flow.source,flow.target,flow.get_name()])
for rel in temp:
    if(isinstance(rel[1],BPMN.Gateway) and len(rel[1].get_in_arcs())>1):
        rel[1]=rel[1].get_out_arcs()[0].target
        while(isinstance(rel[1],BPMN.Gateway)):
            rel[1]=rel[1].get_out_arcs()[0].target

temp1=[]
for rel in temp:
    if(rel[0]==rel[1]):
        continue
    elif(isinstance(rel[0],BPMN.Gateway) and len(rel[0].get_in_arcs())>1):
        continue
    else:
        temp1.append(rel)
df_BPMN_event = pd.DataFrame(columns=['Source', 'Target', 'Relation'])
for rel in temp1:
    if(isinstance(rel[1],BPMN.EndEvent) or isinstance(rel[0],BPMN.StartEvent)):
        continue
    if(isinstance(rel[0],BPMN.ExclusiveGateway)):
        # temp2.append([rel[0].get_in_arcs()[0].source, rel[1]])
        df_BPMN_event = df_BPMN_event.append({'Source': rel[0].get_in_arcs()[0].source.name, 'Target': rel[1].name, 'Relation': 'XOR_gateway'}, ignore_index=True)
        df_BPMN_event = df_BPMN_event.append({'Source': rel[2], 'Target': rel[1].name, 'Relation': 'XOR_condition'}, ignore_index=True)
        # print([rel[0].get_in_arcs()[0].source, rel[1]])
    elif(isinstance(rel[1],BPMN.ExclusiveGateway)):
        continue
    elif(isinstance(rel[0],BPMN.ParallelGateway)):
        df_BPMN_event = df_BPMN_event.append({'Source': rel[0].get_in_arcs()[0].source.name, 'Target': rel[1].name, 'Relation': 'parallel_gateway'}, ignore_index=True)
    elif(isinstance(rel[1],BPMN.ParallelGateway)):
        continue
    else:
        df_BPMN_event = df_BPMN_event.append({'Source': rel[0].name, 'Target': rel[1].name, 'Relation': 'Flow'}, ignore_index=True)

parsing log, completed traces ::   0%|          | 0/1000 [00:00<?, ?it/s]

In [49]:
list_model=[]
for row in df_BPMN.itertuples(index=False):
    if(row.Relation!='XOR_condition'):
        list_model.append((row.Source,row.Target,row.Relation))
list_event=[]
for row in df_BPMN_event.itertuples(index=False):
    if(row.Relation!='XOR_condition'):
        list_event.append((row.Source,row.Target,row.Relation))

In [50]:
dif=set(list(set(list_model).difference(set(list_event)))+list(set(list_event).difference(set(list_model))))

In [51]:
precision=len(dif.intersection(dif_gold))/(len(dif))
recall=len(dif.intersection(dif_gold))/len(dif_gold)
print('precision:',precision)
print('recall:',recall)

precision: 1.0
recall: 1.0


In [ ]:
# detected deviation relationships
dif